In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
np.random.seed(1)

import sys
import sklearn
import sklearn.ensemble
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

import shap
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dataset_folder = 'Datasets/drugs/'
train = pd.read_csv(str(dataset_folder+'train.csv'))
test = pd.read_csv(str(dataset_folder+'test.csv'))

In [30]:
df_all = pd.concat([train,test]).reset_index()
del df_all['index']

In [31]:
df_all.replace('', np.nan, inplace=True)
print(df_all.shape)
df_all.dropna(inplace=True)
print(df_all.shape)

(4143, 2)
(4141, 2)


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.model_selection import train_test_split

In [6]:
stops = set(stopwords.words('english'))
not_stop = ["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","wouldn't"]
for i in not_stop:
    if i in stops:
        stops.remove(i)
    
stemmer = SnowballStemmer('english')
def review_to_words(raw_review):
    review_text = raw_review
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if not w in stops]
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    return( ' '.join(stemming_words))

In [32]:
%time df_all['sideEffectsReview'] = df_all['sideEffectsReview'].apply(review_to_words)

Wall time: 2.24 s


In [33]:
def categorize_rating(x):
    if x in ['Mild Side Effects', 'No Side Effects']:
        return 0
    else:
        return 1

In [34]:
df_all['sideEffects'] = df_all["sideEffects"].apply(categorize_rating)

In [35]:
df_train, df_test = train_test_split(df_all, test_size=0.2, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2,
                             max_features = 5000
                            )

In [12]:
%time X_train = vectorizer.fit_transform(df_train['sideEffectsReview'])
%time X_test = vectorizer.transform(df_test['sideEffectsReview'])

Wall time: 69 ms
Wall time: 23.3 ms


In [13]:
y_train = df_train['sideEffects']
y_test = df_test['sideEffects']

In [14]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=3, random_state=1)
c.fit(X_train, y_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(X_train)))
print('Test', sklearn.metrics.accuracy_score(y_test, predict_fn(X_test)))

Train 0.9978864734299517
Test 0.7949336550060314


In [15]:
print(df_all.shape)
df_all = df_all.drop_duplicates()
print(df_all.shape)

(4141, 2)
(3641, 2)


In [36]:
df_train, df_test = train_test_split(df_all, test_size=0.2, random_state=42)

In [37]:
%time X_train = vectorizer.fit_transform(df_train['sideEffectsReview'])
%time X_test = vectorizer.transform(df_test['sideEffectsReview'])

Wall time: 155 ms
Wall time: 27.9 ms


In [38]:
y_train = df_train['sideEffects']
y_test = df_test['sideEffects']

In [39]:
X_trainn = pd.DataFrame(data = X_train.toarray())
X_testt = pd.DataFrame(data = X_test.toarray())

In [40]:
med = X_trainn.median()

In [41]:
explainer = shap.KernelExplainer(predict_fn, med)

In [42]:
samples = 400

In [43]:
%time exp = explainer.shap_values(X_testt[0:samples])
%time exp2 = explainer.shap_values(X_testt[0:samples])

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [02:56<00:00,  3.00it/s]


Wall time: 2min 56s


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [03:10<00:00,  2.28it/s]


Wall time: 3min 10s


In [44]:
i = calc_identity(exp, exp2)
print(i)

(50.0, 200, 400)


In [45]:
sb = calc_stability(exp, y_test.iloc[0:samples])
print(sb)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(187, 400)


In [46]:
s = calc_separability(exp)
print(s)

(1146, 400, 160000, 0.71625)
